## Explorative Datenanalyse (EDA) - Tabellen

Der Begriff "Explorative Datenanalyse" (Explorative Data Analysis: EDA) wird dem Mathematiker John Tukey zugeschrieben. Er charakteristiert EDA als Methode zur Exploration von Datensammlungen unter Verwendung von einfacher Statistik und grafischer Repräsentation um Muster und Regelmäßigkeiten zu entdecken. 

“This is my favourite part about analytics: Taking boring flat data and bringing it to life through visualization “- John Tukey'

Ziele: 
* neue Perspektiven eröffnen
* Einblicke gewinnen
* Hypothesen formulieren

Techniken:
* Plotten von Rohdaten
* Einfache Statistiken

Typen von Daten:
* Bilddaten (z.B. Postkarten)
* Audiodaten (z.B. Interviews)
* Textdaten (z.B. TEI kodierte Dramen)
* Tabellarische Daten (z.B. Metadaten)

### Datenset

Das Datenset, das wir untersuchen, ist *Feeding America: The Historic American Cookbook*. Das Datenset, bestehend aus 76 Kochbüchern, wurde von der Michigan State University Libraries Special Collections zusammengestellt und präsentiert eine repräsentative Auswahl der kulinarischen Geschichte der USA. Das Datenset umfasst einen Zeitraum von 1796 bis 1922. Kochbücher geben uns einen guten Einblick in die wirtschaftliche und soziale Landschaft einer Kultur.  

**Fragestellungen:**
* Welche Zutaten sind aus der Mode gekommen?
* Welche Zutaten sind im 19. Jahrhundert populär geworden? 
* Kann der Einfluss von Einwanderung im Datenset beobachtet werden?  

Das ursprüngliche Datenset besteht aus 76 XML kodierten Dateien, in denen "recipe type", "ingredients", "measurements" und "cooking implements" kodiert sind. 

### Analyse von Feeding America

Die bekannteste Python Bibliothek für die Manipulation und Analyse von tabellarischen Daten ist Pandas (Python Data Analysis Library). Die Bibliothek ist sehr gut geeignet für die explorative Datenanalyse. Zunächst muss Pandas über `pip install pandas` installiert und danach importiert werden. 

In [ ]:
#Pandas über alias importieren
import pandas as pd 

Daten über die Funktion `pandas.read_csv()` einlesen. Die Funktion braucht einen Textstring als Argument, der den Pfad zur Datei "feeding-america.csv" enthält. Die Funktion liefert ein so genanntes DataFrame zurück, das aus Spalten und Zeilen besteht. Das Dataframe wird in der Variable `df` gespeichert. Die Methode `head()`zeigt uns die ersten fünf Zeilen. 

In [ ]:
df = pd.read_csv("data/feeding-america.csv", index_col='date') 
df = df.fillna(0)
df.head(5)

Jede Zeile in dem Dataset repräsentiert ein Rezept aus einem der 76 Kochbücher und enthält unterschiedliche Kategorien. 

Zunächst starten wir mit Basisstatistiken: 
1) Anzahl der Rezepte in der Textsammlung über die Funktion `len()`. Die Funktion len() ist eine built-in Funktion um die Länge bzw. Größe von unterschiedlichen Typen auszugeben, z.B. Textstrings, Listen, etc.

In [ ]:
print(len(df))

2. Die Datei ist in unterschiedliche Rezeptklassen unterteilt, z.B. soups, sweets, vegetable dishes. Um eine Übersicht über alle Kategorien zu bekommen, greifen wir auf die Spalte `recipe_class` zu. 

Über die Methode `unique()` geben wir nur noch die eindeutigen Klassen zurück. 

Um die Frequenz der Verteilung der Klassen zu ermitteln, verwenden wir die Methode `value_counts()`. 

In [ ]:
df['recipe_class'].value_counts().plot(kind='bar', width=0.1)   

Wie können wir die Farben der Bars verändern? 

Matplotlib ist eine Bibliothek zur Erstellung statischer, interaktiver und animierter Visualisierungen. Die Bibliothek wird für eine Vielzahl von Diagrammen und Plots verwendet, z.B. 
* Line plots
* Scatter plots
* Bar charts and histograms
* Pie charts
* Error bars
* Box plots
* Heatmaps

Dokumentation zu Matplotlib: https://matplotlib.org/stable/ 

In [ ]:
import matplotlib.pyplot as plt

Die Methode `groupby()` gruppiert zunächst alle Zeilen aus demselben Jahr in separate DataFrames. Die Größe der Gruppe - in unserem Fall die Anzahl der Rezepte in einem bestimmten Jahr - wird durch die Methode `size()` ermittelt. Schließlich wird die Anzahl über den Aufruf von `plot()` visualisiert. Der Style 'o' gibt Punkte aus. Mehr dazu unter https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html 

In [ ]:
grouped = df.groupby('date')
recipe_counts = grouped.size()
recipe_counts.plot(style='o', xlim=(1810, 1930))

# Wie fügen wir labels hinzu (x- und y-Achse)

### Kulinarische Trends ermitteln

Um zu ermittlen, welche Trends es in der Verwendung von Zutaten gegeben hat, müssen wir zunächst die Zutaten in Listen splitten. Diese werden. Diese werden in der Spalte 'ingredients' angegeben und durch ein Semikolon getrennt. Zur Erinnerung lassen wir uns zunächst die ersten zehn Zeilen ausgeben.

Wie ermitteln wir die erste Zeile aus dem DataFrame? 

In [ ]:
ingredients = df['ingredients'].str.split(';')

Gruppieren aller Zeilen derselben Jahre:

In [ ]:
groups = ingredients.groupby('date')

Führt die Listen aus einem Jahr zusammen: 

In [ ]:
ingredients = groups.sum()

Berechnet die Häufigkeitsverteilungen pro Jahr:

In [ ]:
ingredients = ingredients.apply(pd.Series.value_counts)

Normalisiert die Zahlen indem jede Zutatenzahl durch die Anzahl der Rezepte pro Jahr geteilt wird:

In [ ]:
ingredients = ingredients.divide(recipe_counts, 0)

Das daraus resultierende DataFrame besteht aus Zeilen, die ein bestimmtes Jahr in der Sammlung repräsentieren. Die Spalten repräsentieren die individuellen Zutaten. 

In [ ]:
ingredients.head()

In [ ]:
df.ingredients.unique()

In [ ]:
column_names = ingredients.columns.tolist()
print(column_names)

In [ ]:
ax = ingredients['tomato'].plot(style='o', xlim=(1810,1930))
ax.set_ylabel("Anteil der Zutat in Rezepten")
ax.set_xlabel("Veröffentlichungsjahr")
ax.legend()

In [ ]:
ax = ingredients['tomato'].plot(style='o', xlim=(1810,1930), label='Tomato')
ingredients['avocado'].plot(style='x', label='Avocado') 
ingredients['lemon'].plot(style='s', label='Lemon')
ax.set_ylabel("fraction of recipes")
ax.set_xlabel("year of publication")
ax.legend()

In [ ]:
import scipy.stats
def plot_trend(column, df, line_color='grey', xlim=(1810, 1930)):
    slope, intercept, _, _, _ = scipy.stats.linregress(
        df.index, df[column].fillna(0).values)
    ax = df[column].plot(style='o', label=column)
    ax.plot(
        df.index,
        intercept + slope * df.index,
        '--',
        color=line_color,
        label='_nolengend_')
    ax.set_ylabel("fraction of recipes")
    ax.set_xlabel("year of publication")
    ax.set_xlim(xlim)

    
plot_trend('yeast', ingredients)
plot_trend('baking powder', ingredients)
plot_trend('tomato', ingredients)

Das Beispiel basiert auf Karsdorp, F.; Kestement, M.; Riddell, R. (2021): Humanities Data Analysis. Case Studies with Python.  